In [1]:
%pwd
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [2]:
import time
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from deeplifting.problems import PROBLEMS_BY_NAME
from deeplifting.optimization import (
    run_deeplifting,
    run_differential_evolution,
    run_dual_annealing,
    run_ipopt,
    run_pygranso,
)

In [3]:
problem_names = [
    'ackley',
    'bukin_n6',
    'cross_in_tray',
    'drop_wave',
    'eggholder',
    'griewank',
    'holder_table',
    'levy',
    'levy_n13',
    'rastrigin',
    'schaffer_n2',
    'schaffer_n4',
    'schwefel',
    'shubert',
]

In [4]:
problem_performance_list = []
trials = 5

for problem_name in problem_names:
    print(problem_name)
    # Setup the problem
    problem = PROBLEMS_BY_NAME[problem_name]

    # Get the known minimum
    minimum_value = problem['global_minimum']

    # First run IPOPT
    outputs = run_ipopt(problem, trials=trials)

    # Get the final results for all IPOPT runs
    ipopt_results = pd.DataFrame(
        outputs['final_results'], columns=['x1', 'x2', 'f', 'algorithm', 'time']
    )
    ipopt_results['problem_name'] = problem_name
    ipopt_results['hits'] = np.where(
        np.abs(ipopt_results['f'] - minimum_value) <= 1e-4, 1, 0
    )

    # Add IPOPT to the problem_performance_list
    problem_performance_list.append(ipopt_results)

    # Next add dual annealing
    outputs = run_dual_annealing(problem, trials=trials)

    # Get the final results for all dual annealing runs
    dual_annleaing_results = pd.DataFrame(
        outputs['final_results'], columns=['x1', 'x2', 'f', 'algorithm', 'time']
    )
    dual_annleaing_results['problem_name'] = problem_name
    dual_annleaing_results['hits'] = np.where(
        np.abs(dual_annleaing_results['f'] - minimum_value) <= 1e-4, 1, 0
    )

    # Add dual annealing to the problem_performance_list
    problem_performance_list.append(dual_annleaing_results)

    # Next add differential evolution
    outputs = run_differential_evolution(problem, trials=trials)

    # Get the final results for all differential evolution runs
    differential_evolution_results = pd.DataFrame(
        outputs['final_results'], columns=['x1', 'x2', 'f', 'algorithm', 'time']
    )
    differential_evolution_results['problem_name'] = problem_name
    differential_evolution_results['hits'] = np.where(
        np.abs(differential_evolution_results['f'] - minimum_value) <= 1e-4, 1, 0
    )

    # Add differential evolution to the problem_performance_list
    problem_performance_list.append(differential_evolution_results)

    # Next add pygranso
    outputs = run_pygranso(problem, trials=trials)

    # Get the final results for all differential evolution runs
    pygranso_results = pd.DataFrame(
        outputs['final_results'], columns=['x1', 'x2', 'f', 'algorithm', 'time']
    )
    pygranso_results['problem_name'] = problem_name
    pygranso_results['hits'] = np.where(
        np.abs(pygranso_results['f'] - minimum_value) <= 1e-4, 1, 0
    )

    # Add differential evolution to the problem_performance_list
    problem_performance_list.append(pygranso_results)

    # Finally, add deeplifting
    outputs = run_deeplifting(problem, trials=trials)

    # Get the final results for all differential evolution runs
    deeplifting_results = pd.DataFrame(
        outputs['final_results'], columns=['x1', 'x2', 'f', 'algorithm', 'time']
    )
    deeplifting_results['problem_name'] = problem_name
    deeplifting_results['hits'] = np.where(
        np.abs(deeplifting_results['f'] - minimum_value) <= 1e-4, 1, 0
    )

    # Add differential evolution to the problem_performance_list
    problem_performance_list.append(deeplifting_results)

ackley

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

n_gpus: 0


/Users/ryandevera/.virtualenvs/deeplifing/lib/python3.9/site-packages/pygranso/private/bfgsHessianInverseLimitedMem.py:237: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3575.)
  alpha[j,:]  = self.rho[0,j] * (self.S[:,j].T  @ q)


n_gpus: 0
cpu
bukin_n6
n_gpus: 0
n_gpus: 0
cpu
cross_in_tray
n_gpus: 0
n_gpus: 0
cpu
drop_wave
n_gpus: 0
n_gpus: 0
cpu
eggholder
n_gpus: 0
n_gpus: 0
cpu
griewank
n_gpus: 0
n_gpus: 0
cpu
holder_table
n_gpus: 0
n_gpus: 0
cpu
levy
n_gpus: 0
n_gpus: 0
cpu
levy_n13
n_gpus: 0
n_gpus: 0
cpu
rastrigin
n_gpus: 0
n_gpus: 0
cpu
schaffer_n2
n_gpus: 0
n_gpus: 0
cpu
schaffer_n4
n_gpus: 0
n_gpus: 0
cpu
schwefel
n_gpus: 0
n_gpus: 0
cpu
shubert
n_gpus: 0
n_gpus: 0
cpu


In [5]:
results_df = pd.concat(problem_performance_list, ignore_index=True)

In [6]:
results_df

,x1,x2,f,algorithm,time,problem_name,hits
0,-9.137017e-09,9.521665e-01,2.579928e+00,IPOPT,0.446302,ackley,0
1,-2.164390e-09,9.521665e-01,2.579928e+00,IPOPT,0.023640,ackley,0
2,-4.395006e-09,-4.159192e-09,1.711489e-08,IPOPT,15.176476,ackley,1
3,8.899931e-10,-1.517027e-09,4.974702e-09,IPOPT,15.525436,ackley,1
4,-7.143433e-10,-2.512892e-09,7.389133e-09,IPOPT,14.380002,ackley,1
...,...,...,...,...,...,...,...
345,-8.003211e-01,4.858057e+00,-1.867309e+02,Deeplifting,38.684909,shubert,1
346,-7.083506e+00,-7.708314e+00,-1.867309e+02,Deeplifting,36.494597,shubert,1
347,-7.083506e+00,4.858057e+00,-1.867309e+02,Deeplifting,43.357514,shubert,1
348,5.482864e+00,4.858057e+00,-1.867309e+02,Deeplifting,45.585452,shubert,1


In [7]:
performance_df = (
    results_df.groupby(['problem_name', 'algorithm'])
    .agg({'hits': 'mean', 'time': 'sum'})
    .reset_index()
    .rename(columns={'hits': 'Success Rate', 'time': 'Total Run Time'})
)

In [8]:
performance_df

,problem_name,algorithm,Success Rate,Total Run Time
0,ackley,Deeplifting,1.0,5.291964
1,ackley,Differential Evolution,1.0,1.151019
2,ackley,Dual Annealing,0.4,0.275730
3,ackley,IPOPT,0.6,45.551856
4,ackley,PyGRANSO,0.0,1.406132
...,...,...,...,...
65,shubert,Deeplifting,1.0,213.577507
66,shubert,Differential Evolution,1.0,0.534127
67,shubert,Dual Annealing,1.0,0.245116
68,shubert,IPOPT,0.0,0.884341


In [9]:
performance_df = (
    performance_df.pivot_table(
        index='problem_name',
        columns='algorithm',
        values=['Success Rate', 'Total Run Time'],
    )
    .T.reorder_levels(['algorithm', None])
    .sort_index()
    .T
)

In [10]:
performance_df = performance_df.apply(lambda x: np.round(x, 3))

In [11]:
performance_df.to_csv('/Users/ryandevera/Desktop/deeplifting_results.csv')

In [15]:
performance_df

algorithm      Deeplifting                Differential Evolution  \
              Success Rate Total Run Time           Success Rate   
problem_name                                                       
ackley                 1.0          5.292                    1.0   
bukin_n6               1.0          4.931                    0.0   
cross_in_tray          1.0        117.322                    1.0   
drop_wave              1.0          5.246                    0.8   
eggholder              0.6        210.111                    0.2   
griewank               1.0          4.977                    0.6   
holder_table           0.6        188.783                    1.0   
levy                   1.0          6.162                    1.0   
levy_n13               1.0          5.942                    1.0   
rastrigin              1.0          5.649                    0.8   
schaffer_n2            1.0          5.096                    1.0   
schaffer_n4            0.2        145.129                    1.0   
schwefel               0.8        476.161                    0.6   
shubert                1.0        213.578                    1.0   

algorithm                    Dual Annealing                       IPOPT  \
              Total Run Time   Success Rate Total Run Time Success Rate   
problem_name                                                              
ackley                 1.151            0.4          0.276          0.6   
bukin_n6               0.709            0.0          0.230          0.0   
cross_in_tray          0.143            1.0          0.161          0.8   
drop_wave              0.402            0.2          0.194          0.0   
eggholder              0.300            0.0          0.188          0.0   
griewank               0.772            0.0          0.192          1.0   
holder_table           0.229            1.0          0.177          0.0   
levy                   0.992            0.8          0.180          0.8   
levy_n13               0.980            0.6          0.230          0.6   
rastrigin              0.693            0.4          0.183          0.2   
schaffer_n2            1.173            0.0          0.531          1.0   
schaffer_n4            0.825            0.0          0.387          1.0   
schwefel               0.323            1.0          0.172          0.0   
shubert                0.534            1.0          0.245          0.0   

algorithm                        PyGRANSO                 
              Total Run Time Success Rate Total Run Time  
problem_name                                              
ackley                45.552          0.0          1.406  
bukin_n6              60.704          0.0          1.558  
cross_in_tray          0.173          0.0          0.973  
drop_wave              0.301          0.0          0.305  
eggholder              0.274          0.0          0.975  
griewank               0.143          0.0          0.509  
holder_table           0.185          0.2          1.415  
levy                   0.145          0.2          0.777  
levy_n13              12.855          0.0          1.296  
rastrigin              0.303          0.0          0.720  
schaffer_n2            0.406          0.0        207.504  
schaffer_n4            0.324          0.0        233.699  
schwefel               0.145          0.0          0.609  
shubert                0.884          0.0          2.912